In [1]:
import utility as ut
import numpy as np
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.neural_network import MLPClassifier

In [56]:
# TODO: Process data
# This is easy with dataframes. Pick out what features we wanna use, make sure they are numerical, scale (if necessary)
# Then just pick out the features and go .to_numpy() and then our y values .to_numpy() and we're ready to train

In [3]:
df = ut.make_big_df()
df = ut.process(df)
df

,Offense Score,Defense Score,Drive Number,Play Number,Period,totalseconds,Offense Timeouts,Yard Line,Yards To Goal,Down,...,L2 Play Yards,Play Type,L1 Yards Gained,L2 Yards Gained,L1 Play Type,L2 Play Type,L1 Down,L2 Down,L1 Distance,L1 Distance
0,0,0,1,2,1,893,3.0,75,75,1,...,NaN,1,NaN,NaN,4,4,NaN,NaN,NaN,NaN
1,0,0,1,3,1,870,3.0,75,75,2,...,NaN,3,0.0,NaN,1,4,1.0,NaN,10.0,10.0
2,0,0,1,4,1,840,3.0,71,71,3,...,0.0,1,4.0,0.0,3,1,2.0,1.0,10.0,10.0
3,0,0,1,5,1,833,3.0,61,61,1,...,4.0,1,10.0,4.0,1,3,3.0,2.0,6.0,6.0
4,0,0,1,6,1,827,3.0,61,61,2,...,10.0,1,0.0,10.0,1,1,1.0,3.0,10.0,10.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2077,35,20,22,1,4,162,-5.0,64,36,1,...,0.0,3,44.0,0.0,0,3,4.0,3.0,27.0,27.0
2079,35,20,22,3,4,155,-5.0,63,37,2,...,44.0,3,1.0,44.0,3,0,1.0,4.0,10.0,10.0
2081,35,20,22,5,4,150,-6.0,62,38,3,...,1.0,3,1.0,1.0,3,3,2.0,1.0,11.0,11.0
2083,35,20,22,7,4,96,-6.0,58,42,4,...,1.0,2,1.0,1.0,3,3,3.0,2.0,12.0,12.0


In [61]:
df = ut.make_big_df()
# df['run'] = np.where(df['Play Type'].str.contains('Rush'), 1, 0)
# df['pass'] = np.where(df['Play Type'].str.contains('Pass'), 1, 0)
# df['Scoring'] = df['Scoring'].astype(int)

df[['Clock Minutes', 'Clock Seconds']]
df['totalseconds'] = (df['Clock Minutes'] * 60) + df['Clock Seconds']
df[['Clock Minutes', 'Clock Seconds', 'totalseconds']]
df['pointsscored'] = df['Offense Score'] + df['Defense Score']
df[['Offense Score', 'Defense Score', 'pointsscored']]
df['L1 Yards Gained'] = df.groupby('Game Id')['Yards Gained'].shift()
df['L2 Yards Gained'] = df.groupby('Game Id')['Yards Gained'].shift(2)
df['L1 Play Type'] = df.groupby('Game Id')['Play Type'].shift()
df['L2 Play Type'] = df.groupby('Game Id')['Play Type'].shift(2)
df['L1 Down'] = df.groupby('Game Id')['Down'].shift()
df['L2 Down'] = df.groupby('Game Id')['Down'].shift(2)
df['L1 Down'] = df.groupby('Game Id')['Distance'].shift()
df['L2 Down'] = df.groupby('Game Id')['Distance'].shift(2)
# Down and distance from prev play
df['Play Type'].value_counts()

Play Type
Rush                          252
Pass Reception                167
Pass Incompletion              98
Penalty                        39
Timeout                        34
Kickoff                        33
Punt                           29
Passing Touchdown              22
Kickoff Return (Offense)       17
Rushing Touchdown              11
End Period                     10
Sack                            9
End of Half                     7
Field Goal Good                 7
Field Goal Missed               5
End of Game                     5
Fumble Recovery (Own)           4
Pass Interception Return        2
Fumble Recovery (Opponent)      2
Kickoff Return Touchdown        1
Interception                    1
Safety                          1
Name: count, dtype: int64

In [58]:
key_features = ['Offense Score', 'Defense Score', 'Drive Number', 'Play Number', 'Period', 'totalseconds', 
                'Offense Timeouts', 'Yard Line', 'Yards To Goal', 'Down', 'Distance', 'Previous Play Yards', 
                'Play Type']


# Filter the DataFrame
keep_play_types = ['Rush', 'Pass', 'Rushing Touchdown', 'Pass Reception', 'Pass Incompletion',
                           'Passing Touchdown', 'Pass Interception Return']
df = df[df['Play Type'].isin(keep_play_types)]

play_type_mapping = {
    'Rushing Touchdown': 'Rush',
    'Pass Reception': 'Pass',
    'Pass Incompletion': 'Pass',
    'Passing Touchdown': 'Pass',
    'Pass Interception Return': 'Pass'
}

df = df.replace(play_type_mapping)
df['Play Type'] = LabelEncoder().fit_transform(df['Play Type'])


df = df[key_features]
# df = df.dropna()
X = df.drop(columns=['Play Type']).to_numpy()
y = df['Play Type'].to_numpy()

In [59]:
# TODO: Train models
# Also pretty easy, just input the stuff, tweak hyperparameters (grid search baby) and set that beast loose

In [60]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

clf = RandomForestClassifier()
clf.fit(X_train, y_train)

print(f"Training accuracy: {clf.score(X_train, y_train)}")
print(f"Test accuracy: {clf.score(X_test, y_test)}")

Training accuracy: 1.0
Test accuracy: 0.5185185185185185


# Ensemble of MLPs with boosting

In [ ]:
# load in data

# make model
mlp = MLPClassifier(hidden_layer_sizes=[20], activation='logistic', solver='sgd',
                    alpha=0, batch_size=1, learning_rate_init=.01, shuffle=True,
                    momentum=0, n_iter_no_change=50, max_iter=1000)
clf = mlp.fit(X_train, y_train)

num_itr = clf.n_iter_

# training set acc
train_pred = clf.predict(X_train)
train_acc = accuracy_score(y_train, train_pred)

# test set acc
test_pred = clf.predict(X_test)
test_acc = accuracy_score(y_test, test_pred)

print("Number of Iterations until Convergence:", num_itr)
print("Training Set Accuracy:", train_acc)
print("Test Set Accuracy:", test_acc)